In [1]:
import sys
from os.path import join
gesladir = "/Users/alisonpeard/Documents/Oxford/DPhil/hybridmodels/GeslaDataset"
sys.path.append(gesladir)

# ESL data from Menéndez (2020)

In [2]:
import shapefile
import folium
from itertools import product

esldir = join("4_FLOOD HEIGHT", "Regular Climate")
roi = shapefile.Reader(join(esldir, "REGION_1_E1.shp"))
point0 = roi.shape(0).__geo_interface__['coordinates'][::-1]  # convert to GeoJSON
m=folium.Map(location=point0, zoom_start=5, tiles='OpenStreetMap')  # [28.644800, 77.216721])

xcoords = [roi.bbox[0], roi.bbox[2]]
ycoords = [roi.bbox[1], roi.bbox[3]]
bbox = [i for i in product(ycoords, xcoords)]
bbox.append(bbox[0])
roi.bbox
folium.Rectangle(bbox).add_to(m, name="Region")

# for i, sf in enumerate(roi.iterShapes()):
#     pass

m

# GESLA dataset

In [6]:
from gesla import GeslaDataset

meta_file = join(gesladir, "GESLA3_ALL.csv")
data_path = join(gesladir, "GESLA3/")

g3 = GeslaDataset(meta_file=meta_file, data_path=data_path)

south_lat = 3  # 3.157889
north_lat = 20  # 19.56799
west_lon = 117  # 116.9271
east_lon = 127  # 127.1762

data = g3.load_lat_lon_range(
    south_lat=int(min(ycoords)),
    north_lat=int(max(ycoords)),
    west_lon=int(min(xcoords)),
    east_lon=int(max(xcoords)),
)

# make sure looks like right spot
print(data.site_name.values)

['Legaspi' 'Manila' 'Kota_Kinabalu' 'Tawau' 'Sandakan' 'Davao'
 'Puerto_Princesa' 'Cebu' 'Davao' 'Subic_Bay' 'Jolo' 'Currimao' 'Lubang']


In [56]:
import geopandas as gpd
import numpy as np

#df = data.to_dataframe()
df = df[["sea_level", "latitude", "longitude"]]
df = df.groupby("station").agg(np.mean)  # get average sea-levels
gdf = gpd.GeoDataFrame(
    df, geometry=gpd.points_from_xy(df.longitude, df.latitude))

In [90]:
from folium import plugins

m = folium.Map(location = [15,30], tiles='Cartodb dark_matter', zoom_start = 2)
sea_levels = [[point.latitude, point.longitude, point.sea_level] for _, point in gdf.iterrows()]
plugins.HeatMap(sea_levels).add_to(m)

m

[[13.145, 123.758, -12.171027202056743],
 [14.585, 120.96800000000002, -5.77239908923919],
 [5.983, 116.067, -5.379541739718138],
 [4.233, 117.883, -3.3036133552409144],
 [5.81, 118.067, -0.8945359363990549],
 [7.153, 125.66300000000001, -25.86973440840939],
 [9.75, 118.73299999999999, -11.023132962190878],
 [10.3, 123.917, -15.521797441871787],
 [7.083, 125.633, -6.099905788872463],
 [14.765, 120.252, 0.4260735083900052],
 [6.067, 121.0, -12.264271795139944],
 [17.988, 120.48799999999999, 0.5391035638350475],
 [13.818, 120.20200000000001, 0.7506004412333204]]

In [69]:
from folium import plugins
help(plugins.HeatMap)

Help on class HeatMap in module folium.plugins.heat_map:

class HeatMap(folium.elements.JSCSSMixin, folium.map.Layer)
 |  HeatMap(data, name=None, min_opacity=0.5, max_zoom=18, radius=25, blur=15, gradient=None, overlay=True, control=True, show=True, **kwargs)
 |  
 |  Create a Heatmap layer
 |  
 |  Parameters
 |  ----------
 |  data : list of points of the form [lat, lng] or [lat, lng, weight]
 |      The points you want to plot.
 |      You can also provide a numpy.array of shape (n,2) or (n,3).
 |  name : string, default None
 |      The name of the Layer, as it will appear in LayerControls.
 |  min_opacity  : default 1.
 |      The minimum opacity the heat will start at.
 |  max_zoom : default 18
 |      Zoom level where the points reach maximum intensity (as intensity
 |      scales with zoom), equals maxZoom of the map by default
 |  radius : int, default 25
 |      Radius of each "point" of the heatmap
 |  blur : int, default 15
 |      Amount of blur
 |  gradient : dict, defau